In [7]:
# Imports
import numpy as np
import random as rd
import math
import IOHexperimenter as IOH
from IOHexperimenter import IOH_function
from zipfile import ZipFile
import os
from os.path import basename
import shutil
from shutil import make_archive

#setting the random seed
rd.seed()

In [18]:
class Simple_EA_singlestep_mutate:
    
    def __init__(self, budget):
        
        self.budget = budget
        self.d = 1
                
        # Parameter setting
        self.mu = 15 #number of parents
        self.lamda = 100 #number of offspring
        self.pc = 0.75 #recombination probability
        
    def initialize_individual(self):
        individual = np.empty((self.d,1))
        for i in range (0,self.d):
            individual[i]= rd.uniform(-5, 5)
        return individual

    def initialize_population(self):
        population = np.zeros((self.mu,self.d))
        for k in range(0,self.mu):
            population[k] = np.transpose(self.initialize_individual())
        return population

    def recombine(self,pop):
        offspring = np.zeros((self.d))
        p1 = pop[rd.randrange(0,len(pop))]
        p2 = pop[rd.randrange(0,len(pop))]
        if rd.uniform(0,1) < self.pc:
            for k in range (0,self.d):
                random_number = rd.uniform(0,1)
                if random_number < 0.5:
                    offspring[k] = p1[k]
                else:
                    offspring[k] = p2[k]
        else:
            offspring = p1
        return offspring
    
    def global_discrete_recombine(self, pop):
        offspring = np.zeros((self.d))
        for i in range(0, self.d):
            random_parent = pop[rd.randrange(0,len(pop))]
            offspring[i] = random_parent[i]
            
        return offspring

    def update_sigma(self, sigma, tau):
        new_sigma = sigma * math.exp(tau * np.random.normal(0,1))
        return new_sigma

    def singlestep_mutate (self,ind, sigma):
        result = 0
        mutant = np.zeros(len(ind))
        for j in range (0, len(ind)):
            result = ind[j] + sigma * np.random.normal(0,1)
             # check boundaries first
            if result < -5:
                mutant[j] = -5
            elif result > 5:
                mutant[j] = 5
             # if result is within boundaries, change j-th mutant characteristic to result
            else:
                mutant[j] = result

        return mutant
    
    def update_sigma_vec(self, sigma_vec, global_tau, local_tau):
        updated_sigma = np.zeros(self.d)
        for i in range (0,self.d):
            updated_sigma[i] = sigma_vec[i] * math.exp(global_tau * np.random.normal(0,1) + local_tau * np.random.normal(0,1))

        return updated_sigma

    def indstep_mutate(self, ind, sigma_vec):
        result = 0
        mutant = np.zeros(len(ind))
        for j in range (0,len(ind)):
            result = ind[j] + sigma_vec[j] * np.random.normal(0,1)
            if result < -5:
                mutant[j] = -5
            elif result > 5:
                mutant[j] = 5
                 # if result is within boundaries, change j-th mutant characteristic to result
            else:
                mutant[j] = result
 
        return mutant
    
    def __call__(self, func):
        
        evalcount = 0
        self.d = func.number_of_variables
        eval_budget = self.budget * self.d
        
        sigma = 10/6
        tau = 1 / math.sqrt(self.d)
        #sigma = np.zeros(self.d)
        #for s in range (0,self.d):
        #   sigma[s] = 10 / 6

        #global_tau = 1 / math.sqrt(2*self.d); #for individual stepsize mutation
        #local_tau = 1 / math.sqrt(2*math.sqrt(self.d)); #for individual stepsize mutation
        
        #Initialization
        xp = self.initialize_population() #parent population
        f = np.zeros((self.mu,1)) #parents fitness
        xo = np.zeros((self.lamda,self.d)) #offspring population
        fo = np.zeros((self.lamda,1)) #offspring fitness
        x_opt = None
        f_opt = np.inf
        
        for i in range (0,self.mu):
            f[i] = func(xp[i])
            evalcount = evalcount +1
        
        # Evaluation loop
        while evalcount < eval_budget and func.final_target_hit == False :
            
            #update sigma
            sigma = self.update_sigma(sigma, tau)
            
            
            #create new offspring generation
            for i in range(0,self.lamda):
                offspring = self.recombine(xp)
                xo[i] = self.singlestep_mutate(offspring, sigma)
                fo[i] = func(xo[i])
                evalcount = evalcount +1
                if evalcount == eval_budget:
                    break
                
            # Selection
            m = np.concatenate((xo, fo), axis=1) # Create joint matrix of individuals and their fitness values
            sorted_m = m[np.argsort(m[:, self.d])] # sort based on fitness (lowest values highest ranking)
            
            xpnew = sorted_m[0:self.mu,0:self.d] # copy best individuals and remove fitness values
            xp = xpnew # set new population for next generation
            f = sorted_m[0:self.mu,self.d] # copy fitness values of new generation

            #check if new best individual is better than previous best and copy x and f if true
            if f[0] < f_opt:
                x_opt = xp[0]
                f_opt = f[0]

        return x_opt, f_opt

In [19]:
fIDs = []
for fid in range (1,25):
    fIDs.append(fid)

In [20]:
from IOHexperimenter import IOHexperimenter
exp = IOHexperimenter()
exp.initialize_BBOB(fIDs,[1],[2,5,10], 1)
path = "/Users/dschroeder/Documents/Master Computer Science/Master_Thesis/code_and_data/experiments/"
experiment_ID = input("enter date-time, format ddmm-hhmm")
experiment_name =  "simple_EA/experiment" + experiment_ID
#experiment_parameters = "/pc" + "95"
exp.set_logger_location(path + experiment_name, "run")
exp([Simple_EA_singlestep_mutate(5000)])
#create a ZIP file for IOHprofiler upload
shutil.make_archive(path + experiment_name,"zip", path + experiment_name)

enter date-time, format ddmm-hhmm 0809-1629


Running single-threaded
1 7115 2.1863143408914798e-10
1 21415 3.874319691812514e-09
1 50000 2.6212816326201263
2 10000 0.07314222002305983
2 25000 2.502993278162245
2 50000 378.464229458251
3 10000 1.2658399231196602
3 25000 9.483437918834715
3 50000 71.75736214851439
4 10000 1.8229848003533543e-08
4 25000 1.9899281633739252
4 50000 4.974805124664288
5 215 0.0
5 715 0.0
5 15615 0.0
6 10000 1.2641381139280168
6 25000 0.027208753651227893
6 50000 3.532730230948426
7 10000 92.9400000041737
7 25000 93.40633654086996
7 50000 146.07407432027577
8 10000 0.10837924423670725
8 25000 5.79236986952056
8 50000 1247.7937649015157
9 10000 7.522653914368932e-05
9 25000 79.73798572036823
9 50000 7.057309935685111
10 10000 40.72648878523003
10 25000 4.989665580722283
10 50000 510.3525626421668
11 10000 44.815626646647814
11 25000 7.310270359425031
11 50000 101.49964862808126
12 10000 0.8240598260204088
12 25000 37.37476164303199
12 50000 33899.14421719119
13 10000 0.00021542621515692573
13 25000 6.9472

'/Users/dschroeder/Documents/Master Computer Science/Master_Thesis/code_and_data/experiments/simple_EA/experiment0809-1629.zip'